### Extract SEC Filing Archives

In [ ]:
import os

def create_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
import zipfile

def extract_archive(archive, directory):
    with zipfile.ZipFile(archive, 'r') as zip_ref:
        zip_ref.extractall(directory)

In [ ]:
import os
import progressbar

sec_dir = 'sec_filings'
for item in progressbar.progressbar(sorted(os.listdir(f'{sec_dir}/zips'))):
    if item.startswith('20'):
        quarter = item.split('.')[0]
        quarter_directory = f'{sec_dir}/{quarter}'
        create_dir(quarter_directory)
        extract_archive(f'{sec_dir}/zips/{item}', quarter_directory)

### Extract Fundamental Data

In [1]:
import pandas as pd
import os
import progressbar

sec_dir = 'sec_filings'
values = ['CommonStockSharesOutstanding', 'EarningsPerShareBasic']
quarters = [
    quarter
    for quarter in os.listdir(sec_dir)
    if quarter.startswith('20')
]
df_list = []
used_quarters = []
for quarter in progressbar.progressbar(quarters):  
    sub_df = pd.read_csv(f'{sec_dir}/{quarter}/sub.txt', delimiter='\t')
    if sub_df.shape[0] == 0:
        continue
    num_df = pd.read_csv(f'{sec_dir}/{quarter}/num.txt', delimiter='\t')
    # need to understand how ddate works here
    num_df = num_df.sort_values('ddate')
    num_df = num_df[['adsh', 'tag', 'ddate', 'value']].drop_duplicates(
        subset=['adsh', 'tag'], keep='last'
    )
    sub_df.loc[:, 'symbol'] = sub_df.instance.apply(
        lambda x: x.split('-')[0].upper()
    )
    sub_df = sub_df.sort_values('filed').drop_duplicates(subset=['symbol'], keep='last') #duplicate adsh values found
    sub_df.loc[:, 'year'] = int(quarter[:4])
    sub_df.loc[:, 'quarter'] = int(quarter[5])
    for index, value in enumerate(values):
        value_df = num_df.loc[num_df.tag == value][['adsh', 'value']]
        value_df = value_df.rename(columns={"value": value})
        sub_df = sub_df.merge(
            value_df, on='adsh',
            how=('inner' if (index + 1) == len(values) else 'left')
        )
    df_list.append(sub_df[values + ['symbol', 'year', 'quarter']])
    used_quarters.append(quarter)
fundamental_df = pd.concat(df_list, axis=0, ignore_index=True)
fundamental_df.to_parquet('fundamentals.parquet')

 23% (11 of 47) |#####                   | Elapsed Time: 0:00:39 ETA:   0:01:49/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 74% (35 of 47) |#################       | Elapsed Time: 0:02:16 ETA:   0:01:13/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100% (47 of 47) |########################| Elapsed Time: 0:03:01 Time:  0:03:01
